<a href="https://colab.research.google.com/github/AlyaaBajaber/Sadai_T5/blob/main/Copy_of_YOLO_Practical_Exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Road Traffic Vehicle Counting Exam

In this exam, you will use the YOLO (You Only Look Once) object detection model to analyze a video of road traffic. Your primary task is to count the number of vehicles exiting the road at each of the four exits (located at the top, bottom, left, and right of the frame). The video for this task is provided at `Datasets/YOLO_Exam_Video.mp4`.

## Objectives
- Load and process the video using OpenCV.
- Utilize the YOLO model to detect vehicles in each frame.
- Manipulate the video frames using OpenCV.
- Track vehicles as they move across the video frames.
- Count the number of vehicles exiting via the top street.
- Count the number of vehicles exiting via the bottom street.
- Count the number of vehicles exiting via the left street.
- Count the number of vehicles exiting via the right street.


## Setup Environment

Before you begin, ensure you have the necessary libraries installed. You will need `opencv`, and `ultralytics` among others.
If these are not installed, you should install them.

In [2]:
# prompt: library opencv

#!pip install opencv-python


In [3]:
#!pip install ultralytics
#!pip install opencv-p

## Load the YOLO Model

You will first need to load the YOLO model, there is a custom model for this exam called `YOLO_Model.pt` under Datasets file ⚠️ PLEASE USE THIS MODEL AND DO NOT USE ANY OTHER MODEL ⚠️.
Write the code to load the YOLO model below:

In [4]:
import cv2
import numpy as np
from ultralytics import YOLO, solutions

In [5]:
#model = YOLO('/content/YOLO_Model.pt')

In [6]:
model = YOLO('yolov5s.pt')

PRO TIP 💡 Replace 'model=yolov5s.pt' with new 'model=yolov5su.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.



## Prepare the Video Capture

Create a variable to capture the video frames, you can use `cv2.VideoCapture()` to achive this.

In [7]:
cap = cv2.VideoCapture('/content/YOLO_Video.mp4')
assert cap.isOpened(), 'Error reading video file'
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

## Get Video Information

You can use `cv2` library to get these information fro the `VideoCapture()` variable you created to extract these information:
* `height`: Video's height.
* `width`: Video's width.
* `fps`: Video's frames.

In [8]:
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
fps = cap.get(cv2.CAP_PROP_FPS)

    # Print the video information
print(f"Video Height: {height}")
print(f"Video Width: {width}")
print(f"Frames per Second (FPS): {fps}")

Video Height: 1080
Video Width: 1920
Frames per Second (FPS): 29.97002997002997


## Prepare Video Writer to Store the Output

Create a variable that uses `cv2.VideoCapture()` to save the video with the bounding boxes, specified Region of Interest rectangle (ROI) and the counted vehicles on each sides. You will need to make the video with the same `fps`, `width`, `height`, and specify the codec and output path of the video.

In [9]:
video_writer = cv2.VideoWriter('test_output.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))

In [10]:
# Assuming w and h are the width and height of the frame
mid_x = w // 4  # Quarter point of the width
mid_y = h // 4  # Quarter point of the height

# Define line points
line_points1 = [(2, mid_y), (w - 2, mid_y)]           # Horizontal line at quarter height
line_points2 = [(40, mid_y), (w - 40, mid_y)]       # Horizontal line at quarter height with margins
line_points3 = [(mid_x, 30), (mid_x, h - 30)]           # Vertical line at quarter width
line_points4 = [(80, mid_y), (w - 80, mid_y)]         # Horizontal line with 80 pixels margin from sides

In [11]:
ret, frame = cap.read()

In [12]:
# Inside your video processing loop
cv2.line(frame, line_points1[0], line_points1[1], (255, 0, 0), 2)  # Line 1 in red
cv2.line(frame, line_points2[0], line_points2[1], (0, 255, 0), 2)  # Line 2 in green
cv2.line(frame, line_points3[0], line_points3[1], (0, 0, 255), 2)  # Line 3 in blue
cv2.line(frame, line_points4[0], line_points4[1], (255, 255, 0), 2)  # Line 4 in cyan

array([[[ 41,  77,  72],
        [ 54,  90,  85],
        [ 38,  79,  73],
        ...,
        [ 55,  51,  47],
        [ 55,  51,  47],
        [ 55,  51,  47]],

       [[ 22,  58,  53],
        [ 49,  85,  80],
        [ 33,  74,  68],
        ...,
        [ 56,  52,  48],
        [ 55,  51,  47],
        [ 55,  51,  47]],

       [[  6,  42,  37],
        [ 40,  76,  71],
        [ 34,  75,  69],
        ...,
        [ 55,  51,  47],
        [ 55,  51,  47],
        [ 54,  50,  46]],

       ...,

       [[128, 118, 194],
        [111, 101, 177],
        [ 99,  87, 163],
        ...,
        [108, 117, 123],
        [108, 117, 123],
        [108, 117, 123]],

       [[117, 100, 183],
        [101,  84, 167],
        [112, 100, 164],
        ...,
        [110, 119, 125],
        [108, 117, 123],
        [108, 117, 123]],

       [[ 99,  82, 165],
        [106,  89, 172],
        [126, 114, 178],
        ...,
        [110, 119, 125],
        [110, 119, 125],
        [110, 119, 125]]

## Expermint to get the `(x1, y1, x2, y2)` of the ROI

### Hint
When defining the Regions of Interest (ROIs) for detecting vehicles exiting the road, consider the coordinates that outline the relevant areas. These coordinates can be specified as `(x1, y1, x2, y2)`, where `(x1, y1)` is the top-left corner and `(x2, y2)` is the bottom-right corner of the rectangle.

Experiment with different coordinates to accurately cover the streets where vehicles exit. Visualize these ROIs by drawing rectangles on a frame of the video and adjust the coordinates as needed until you capture the desired areas effectively.

In [14]:
# Define ROIs for vehicle exits
roi_top = (0, 0, width, height // 10)  # Top exit
roi_bottom = (0, height - height // 10, width, height)  # Bottom exit
roi_left = (0, 0, width // 10, height)  # Left exit
roi_right = (width - width // 10, 0, width, height)  # Right exit

## Process Video Frames and Identify Vehicles Left the Road Form Each Street

For each frame in the video, use the YOLO model to detect and track vehicles. You will need to write a loop that processes each frame and applies the YOLO model.

In each frame, after detecting the vehicles, determine whether they exited the road via the top, bottom, left, or right street. You can use the positions of the bounding boxes provided by YOLO to do this.

- The video should display bounding boxes around the detected objects.
- The video should display your name on the top.
- It should display the calculated center of each vehicle.
- The video should display the confidence score, alongside the object ID and class ID of each detected and tracked object.
- The video should display the rectangles representing the regions where you count the vehicles that have crossed and exited.
- The video should display the number of vehicles that exited via the top street.
- The video should display the number of vehicles that exited via the bottom street.
- The video should display the number of vehicles that exited via the left street.
- The video should display the number of vehicles that exited via the right street.

In [15]:
counter1 = solutions.ObjectCounter(
    view_img=True,
    reg_pts=None,
    names=model.names,
    draw_tracks=True,
    line_thickness=2,
)
'''
counter2 = solutions.ObjectCounter(
    view_img=True,
    reg_pts=line_points2,
    names=model.names,
    draw_tracks=True,
    line_thickness=2,
)
counter3 = solutions.ObjectCounter(
    view_img=True,
    reg_pts=line_points3,
    names=model.names,
    draw_tracks=True,
    line_thickness=2,
)
counter4 = solutions.ObjectCounter(
    view_img=True,
    reg_pts=line_points4,
    names=model.names,
    draw_tracks=True,
    line_thickness=2,
)
'''

WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()

Line Counter Initiated.


'\ncounter2 = solutions.ObjectCounter(\n    view_img=True,\n    reg_pts=line_points2,\n    names=model.names,\n    draw_tracks=True,\n    line_thickness=2,\n)\ncounter3 = solutions.ObjectCounter(\n    view_img=True,\n    reg_pts=line_points3,\n    names=model.names,\n    draw_tracks=True,\n    line_thickness=2,\n)\ncounter4 = solutions.ObjectCounter(\n    view_img=True,\n    reg_pts=line_points4,\n    names=model.names,\n    draw_tracks=True,\n    line_thickness=2,\n)\n'

In [17]:
while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print('Video frame is empty or video processing has been successfully completed.')
        break

    # Perform object tracking
    tracks = model.track(im0, persist=True, show=False)

    # Count objects and draw the middle line
    im0 = counter1.start_counting(im0, tracks)

    #im02 = counter2.start_counting(im01, tracks)

   # im03= counter3.start_counting(im02, tracks)

  #  im04 = counter4.start_counting(im03, tracks)
    video_writer.write(im0)


0: 384x640 1 person, 8 cars, 343.8ms
Speed: 4.0ms preprocess, 343.8ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 5 cars, 339.7ms
Speed: 3.9ms preprocess, 339.7ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 6 cars, 1 truck, 1 traffic light, 333.9ms
Speed: 3.8ms preprocess, 333.9ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 3 cars, 1 truck, 337.2ms
Speed: 4.6ms preprocess, 337.2ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 5 cars, 1 train, 354.7ms
Speed: 3.7ms preprocess, 354.7ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 6 cars, 330.6ms
Speed: 4.2ms preprocess, 330.6ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 8 cars, 1 traffic light, 355.9ms
Speed: 3.6ms preprocess, 355.9ms inference, 1.6ms postprocess per image at shape (1, 

In [18]:
cap.release()
video_writer.release()
cv2.destroyAllWindows()

## Save and Submit Your Work

Submit both the notebook and the output video